<a href="https://colab.research.google.com/github/O-Kpy/Dacon/blob/main/Dacon_%EC%8B%A0%EC%9A%A9%EC%98%88%EC%B8%A1_%ED%95%84%EC%82%AC_%ED%95%98%EC%9D%B4%ED%8D%BC%ED%8C%8C%EB%9D%BC%EB%AF%B8%ED%84%B0_%EC%B5%9C%EC%A0%81%ED%99%94%ED%95%B4%EB%B3%B4%EA%B8%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [43]:
import pandas as pd
import numpy as np

from catboost import CatBoostClassifier
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.metrics import log_loss
from tensorflow.keras.utils import to_categorical

import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')
from IPython.core.display import display, HTML
display(HTML("<style>.container { width: 100% !important; }</style>"))

In [44]:
train = pd.read_csv('/content/drive/MyDrive/dataset/Dacon/대회/신용카드 사용자 연체 예측 AI 경진대회/train.csv')
test = pd.read_csv('/content/drive/MyDrive/dataset/Dacon/대회/신용카드 사용자 연체 예측 AI 경진대회/test.csv')
submit = pd.read_csv('/content/drive/MyDrive/dataset/Dacon/대회/신용카드 사용자 연체 예측 AI 경진대회/sample_submission.csv')
test_index = test['index']
train = train.drop(['index'], axis=1)
test = test.drop(['index'], axis=1)
print(f'train shape:{train.shape}')
print(f'test shape:{test.shape}')


train shape:(26457, 19)
test shape:(10000, 18)


In [5]:
!pip install catboost

     |████████████████████████████████| 67.3MB 64kB/s 


In [45]:
def days_to_year(x):
  return (x * -1) / 365

def minus(x):
  return x * -1

# 이상치 제거

In [46]:
def remove_outlier(train, column):
    
    df = train[column]
    
    # 1분위수
    quan_25 = np.percentile(df.values, 25)
    
    # 3분위수
    quan_75 = np.percentile(df.values, 75)
    
    iqr = quan_75 - quan_25
    
    lowest = quan_25 - iqr * 5
    highest = quan_75 + iqr * 5
    outlier_index = df[(df < lowest) | (df > highest)].index
    print('outlier의 수 : ' , len(outlier_index))
    print(df.iloc[outlier_index])
    train.drop(outlier_index, axis = 0, inplace = True)
    
    return train

candidate = ['child_num']
for cand in candidate:  
    train = remove_outlier(train,cand)

train.reset_index(drop = True,inplace = True)
len(train)

outlier의 수 :  6
8462     14
9021     14
10731    19
25313     7
25390    14
25638     7
Name: child_num, dtype: int64


26451

# 중복방지 이건 나중에 따로

In [47]:
def add_var(data):
    
    # 개개인을 구분할 수 있는 변수들을 묶어서 생성
    data['personal_id'] = data['gender'] + "_" + data['DAYS_BIRTH'].astype(str) + "_" + data['income_total'].astype(str) + "_" + data['income_type'].astype(str)
    
    # 카드를 생성한 기간도 같은 경우가 있어서 begin을 추가하여 하나의 변수를 더 생성
    data['personal_begin_id'] = data['gender'] + "_" + data['DAYS_BIRTH'].astype(str) + "_" + data['income_total'].astype(str) + "_" + data['income_type'].astype(str) + "_" + data['begin_month'].astype(str)
    
    # 그외의 변수들을 조합하여 하나의 변수로 추가 생성
    data['g_r_c'] = data['gender'] + "_" + data['reality'] + "_" + data['car'] 
    data['p_w_e'] = data['phone'].astype(str) + "_" + data['work_phone'].astype(str) + "_" + data['email'].astype(str) 
    
    return data

train = add_var(train)
test = add_var(test)

# 숫자형 변수 전처리 및 변수 생성

In [48]:
def numeric_process(data):
    
    # income_total 변수 전처리
    # 만단위로 생성
    data['income_total'] = data['income_total']/10000
    # 편차 제곱 변수 생성
    data['income_total_dev'] = (data['income_total'] - data['income_total'].mean())**2
    # 로그 변환
    data['income_total_log'] = data['income_total'].apply(np.log1p)

    # DAYS_EMPLOYED 변수 전처리
    # 0 이상인 경우 0으로 모두 변환
    data.loc[data['DAYS_EMPLOYED'] >= 0,'DAYS_EMPLOYED']=0
    # day를 year로 변환
    data['DAYS_EMPLOYED'] = data['DAYS_EMPLOYED'].apply(days_to_year) 
    # 로그 변환
    data['DAYS_EMPLOYED_log'] = data['DAYS_EMPLOYED'].apply(np.log1p)

    # begin_month 변수 전처리
    # 마이너스 변환
    data['begin_month'] = data['begin_month'].apply(minus)
    
    # DAYS_BIRTH 변수 전처리
    # day를 year로 변환
    data['DAYS_BIRTH'] = data['DAYS_BIRTH'].apply(days_to_year)

    #  DAYS_BIRTH, DAYS_EMPLOYED, income_total변수를 조합하여 RATIO 변수 생성
    data['EMPLOYED_BIRTH_RATIO'] = data['DAYS_EMPLOYED']/data['DAYS_BIRTH']
    data['INCOME_EMPLOYED_RATIO'] = data['income_total']/data['DAYS_EMPLOYED']
    data['INCOME_BIRTH_RATIO'] = data['income_total']/data['DAYS_BIRTH']
    
    # 가족수 - 자식수
    data['diff_fam_child'] = data['family_size'] - data['child_num']
    # chid_num과 family_size는 다음과 같이 최대 2와 5가 되도록 전처리
    data.loc[data['child_num'] >= 2,'child_num'] = 2
    data.loc[data['family_size'] >= 5,'child_num'] = 5
    # 가족수와 자녀수 sum 변수 추가
    data['FAM_CHILD_SUM'] = data[['child_num', 'family_size']].sum(axis=1)
    
    # income을 가족 수 및 자식 수로 나눈 비율
    data['INCOME_FAM_RATIO'] = data['income_total']/data['family_size']
    data['INCOME_child_num_RATIO'] = data['income_total']/data['child_num']
    
    # 일을하게 된 시점 변수 추가
    data['BIRTH_MINUS_EMPLOYED'] = data['DAYS_BIRTH'] - data['DAYS_EMPLOYED']
    # income total 변수에 before_EMPLOYED로 나눈 변수 추가
    data['INCOME_BIRTH_MINUS_EMPLOYED_RATIO'] = data['income_total']/data['BIRTH_MINUS_EMPLOYED']
    
    return data 


In [49]:
train = numeric_process(train)
test = numeric_process(test)

In [50]:
# occyp_type 변수 전처리

def occype_process(data):

  data['occyp_type'] = data['occyp_type'].fillna('NAN')
  # 경력이 없고 직업군이 none인 사람은 no_work로 대체
  data.loc[(data['DAYS_EMPLOYED'] == 0) & (data['occyp_type'] == 'NAN'), 'occyp_type'] = 'no_work'
  print(data['occyp_type'].value_counts(), '\n\n')

  return data

train = occype_process(train)
test = occype_process(test)

Laborers                 4512
no_work                  4438
NAN                      3733
Core staff               2646
Sales staff              2539
Managers                 2167
Drivers                  1572
High skill tech staff    1040
Accountants               902
Medicine staff            864
Cooking staff             457
Security staff            424
Cleaning staff            401
Private service staff     243
Low-skill Laborers        127
Waiters/barmen staff      123
Secretaries                97
Realty agents              63
HR staff                   62
IT staff                   41
Name: occyp_type, dtype: int64 


Laborers                 1699
no_work                  1697
NAN                      1455
Sales staff               946
Core staff                945
Managers                  845
Drivers                   563
Medicine staff            343
High skill tech staff     343
Accountants               339
Cooking staff             198
Security staff            168
Cleani

In [51]:
#구간화 함수
def make_bin(df, variable, n):
    
    data = df
    count, bin_dividers = np.histogram(data[variable], bins=n)
    bin_names=[str(i) for i in range(n)]
    data['%s_bin' % variable] = pd.cut(x=data[variable], bins=bin_dividers, labels=bin_names, include_lowest=True)
    data['%s_bin' % variable] = pd.factorize(data['%s_bin' % variable])[0]
    print(data['%s_bin' % variable], '\n\n')
    
    return data

# days_birth만 구간화 했을 떄 가장 성능이 좋았음
train = make_bin(train, 'DAYS_BIRTH', n=10)
test = make_bin(test, 'DAYS_BIRTH', n=10)

0        0
1        1
2        2
3        3
4        3
        ..
26446    1
26447    3
26448    5
26449    5
26450    2
Name: DAYS_BIRTH_bin, Length: 26451, dtype: int64 


0       0
1       1
2       2
3       1
4       3
       ..
9995    1
9996    8
9997    4
9998    3
9999    9
Name: DAYS_BIRTH_bin, Length: 10000, dtype: int64 




In [52]:
# feature selection
train = train.drop(['income_total', 'DAYS_EMPLOYED', 'FLAG_MOBIL'], axis=1)
test = test.drop(['income_total', 'DAYS_EMPLOYED', 'FLAG_MOBIL'], axis=1)

In [53]:
print(train.shape, test.shape)

(26451, 33) (10000, 32)


# 데이터 분할

> 컬럼 순서대로 정렬 후 object형 변수의 인덱스 정보를 cat_feature에 활용

In [54]:
x_train = train.drop(['credit'], axis=1)
y_train = train['credit']
x_test = test

In [55]:
cat_features = [f for f in x_train.columns if x_train[f].dtype=='object']

def columns_index(df, cat_features):
  cols = df.columns.values
  sidx = np.argsort(cols)
  return sidx[np.searchsorted(cols, cat_features, sorter=sidx)]

cat_features_idx = columns_index(x_train, cat_features)
print('Cat features are:', [f for f in cat_features])
print(cat_features_idx)

Cat features are: ['gender', 'car', 'reality', 'income_type', 'edu_type', 'family_type', 'house_type', 'occyp_type', 'personal_id', 'personal_begin_id', 'g_r_c', 'p_w_e']
[ 0  1  2  4  5  6  7 12 15 16 17 18]


In [56]:
cat_models = {}

skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=40)
outcomes =[]
sub=np.zeros((x_test.shape[0], 3))

for n_fold, (train_index, test_index) in enumerate(skf.split(x_train, y_train)):
  X_train, X_valid = x_train.iloc[train_index], x_train.iloc[test_index]
  Y_train, Y_valid = y_train.iloc[train_index], y_train.iloc[test_index]

  cat = CatBoostClassifier(n_estimators=3000, max_depth=8, random_seed = 1042, learning_rate=0.04, bootstrap_type='Bernoulli')
  cat.fit(X_train, Y_train, eval_set=[(X_train, Y_train), (X_valid, Y_valid)], early_stopping_rounds = 50, cat_features=cat_features, verbose=100)

  cat_models[n_fold] = cat

  # val 데이터 예측
  predict = cat.predict_proba(X_valid)
  # test 데이터 예측
  test_predict = cat.predict_proba(x_test)

  # val 데이터 예측 logloss 값 저장
  logloss=log_loss(to_categorical(Y_valid), predict)
  outcomes.append(logloss)
  print(f'FOLD{n_fold+1}:logloss:{logloss}')

  # test 데이터 예측 결과 종합
  # 최종적으로는 kfolds 횟수 만큼 나눠서 평균 값을 활용
  sub+=test_predict

  print(f'-----------------------------------------------------------------------------------------------------------------\n\n')

mean_outcome = np.mean(outcomes)
print('Mean:{}'.format(mean_outcome))
sub=sub/(folds.n_splits * len(random_seed))




0:	learn: 1.0754903	test: 1.0754903	test1: 1.0752126	best: 1.0752126 (0)	total: 62.9ms	remaining: 3m 8s
100:	learn: 0.7160559	test: 0.5165827	test1: 0.6697389	best: 0.6697389 (100)	total: 24.9s	remaining: 11m 55s
200:	learn: 0.7000987	test: 0.4942297	test1: 0.6620249	best: 0.6620249 (200)	total: 56.3s	remaining: 13m 3s
300:	learn: 0.6839706	test: 0.4853135	test1: 0.6600025	best: 0.6600025 (300)	total: 1m 30s	remaining: 13m 33s
400:	learn: 0.6690772	test: 0.4795943	test1: 0.6593254	best: 0.6593077 (397)	total: 2m 5s	remaining: 13m 33s
500:	learn: 0.6531109	test: 0.4755854	test1: 0.6584701	best: 0.6584701 (500)	total: 2m 40s	remaining: 13m 18s
600:	learn: 0.6358093	test: 0.4721645	test1: 0.6578014	best: 0.6578014 (600)	total: 3m 14s	remaining: 12m 57s
Stopped by overfitting detector  (50 iterations wait)

bestTest = 0.6577942
bestIteration = 601

Shrink model to first 602 iterations.
FOLD1:logloss:0.6577941999822433
------------------------------------------------------------------------

KeyboardInterrupt: ignored

In [59]:
pip install optuna

     |████████████████████████████████| 296kB 3.6MB/s 
     |████████████████████████████████| 81kB 3.7MB/s 
     |████████████████████████████████| 174kB 4.7MB/s 
     |████████████████████████████████| 51kB 3.2MB/s 
     |████████████████████████████████| 143kB 5.3MB/s 
     |████████████████████████████████| 112kB 5.5MB/s 
     |████████████████████████████████| 81kB 4.4MB/s 
  Created wheel for pyperclip: filename=pyperclip-1.8.2-cp37-none-any.whl size=11107 sha256=6935f85bb7119dfddcfdcb9658deb3094ab115e886ae7e4b191fedd0301f4ac4
  Stored in directory: /root/.cache/pip/wheels/25/af/b8/3407109267803f4015e1ee2ff23be0c8c19ce4008665931ee1
Successfully built pyperclip


In [99]:
import optuna
from lightgbm import LGBMClassifier
from optuna import Trial
from optuna.samplers import TPESampler

def objective(trial: Trial) -> float:
    params_cat = {
        "random_seed": trial.suggest_int('random_seed', 42, 1043),
        "learning_rate": trial.suggest_float('learning_rate', 0.01, 0.04),
        "n_estimators": 3000,
        "reg_lambda": trial.suggest_float("reg_lambda", 1e-8, 9e-2),
        "max_depth": trial.suggest_int("max_depth", 1, 20),
        "colsample_bylevel": trial.suggest_float("colsample_bylevel", 0.4, 1.0),
        "subsample": trial.suggest_float("subsample", 0.3, 1.0),
        "min_child_samples": trial.suggest_int("min_child_samples", 5, 100),
        "max_bin": trial.suggest_int("max_bin", 200, 500),
        'bootstrap_type':'Bernoulli'
        
    }
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=40)
    for i, (train_indice, valid_indice) in enumerate(skf.split(X=x_train, y=y_train)):
      X_train, X_valid = x_train.iloc[train_indice], x_train.iloc[valid_indice]
      Y_train, Y_valid = y_train.iloc[train_indice], y_train.iloc[valid_indice]

      model = CatBoostClassifier(**params_cat)
      model.fit(
          X_train,
          Y_train,
          eval_set=[(X_train, Y_train), (X_valid, Y_valid)],
          early_stopping_rounds=100,
          verbose=False,
          cat_features=cat_features
          )

    lgb_pred = model.predict_proba(X_valid)
    log_score = log_loss(Y_valid, lgb_pred)
    
    return log_score

In [100]:
sampler = TPESampler(seed=42)
study = optuna.create_study(
    study_name="cat_parameter_opt",
    direction="minimize",
    sampler=sampler,
)
study.optimize(objective, n_trials=10)
print("Best Score:", study.best_value)
print("Best trial:", study.best_trial.params)

[I 2021-05-25 12:44:28,749] A new study created in memory with name: cat_parameter_opt


KeyboardInterrupt: ignored